In [42]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from PIL import Image
import time
import os

def read_binary_file(file_path):
    with open(file_path, 'rb') as file:
        binary_data = file.read()
    binary_string = ''.join(format(byte, '08b') for byte in binary_data)
    return binary_string

def encode_binary_image(cover_image_path, binary_message, output_image_path):
    start_time = time.time()
    img = Image.open(cover_image_path)
    width, height = img.size
    pixels = np.array(img, dtype=np.uint8)
    
    # Encode the length of the binary message first (fixed 32 bits for simplicity)
    message_length = len(binary_message)
    length_binary = format(message_length, '032b')  # 32 bits for the length
    full_binary = length_binary + binary_message
    
    full_binary = np.array([int(bit) for bit in full_binary], dtype=np.uint8)
    
    mod = SourceModule("""
    __global__ void encode_kernel(unsigned char *pixels, unsigned char *full_binary, int message_length) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        int bit_idx = idx * 4;
        if (bit_idx < message_length) {
            for (int n = 0; n < 3; ++n) {
                unsigned char channel = pixels[idx * 3 + n];
                unsigned char mask = 0xF0;
                unsigned char new_bits = (full_binary[bit_idx + n * 4 + 0] << 3) |
                                         (full_binary[bit_idx + n * 4 + 1] << 2) |
                                         (full_binary[bit_idx + n * 4 + 2] << 1) |
                                         full_binary[bit_idx + n * 4 + 3];
                pixels[idx * 3 + n] = (channel & mask) | new_bits;
            }
        }
    }
    """)
    
    pixels_gpu = cuda.mem_alloc(pixels.nbytes)
    binary_gpu = cuda.mem_alloc(full_binary.nbytes)
    
    cuda.memcpy_htod(pixels_gpu, pixels)
    cuda.memcpy_htod(binary_gpu, full_binary)
    
    block_size = 512
    grid_size = (width * height + block_size - 1) // block_size
    
    encode_kernel = mod.get_function("encode_kernel")
    encode_kernel(pixels_gpu, binary_gpu, np.int32(len(full_binary)), block=(block_size, 1, 1), grid=(grid_size, 1, 1))
    
    cuda.memcpy_dtoh(pixels, pixels_gpu)
    
    encoded = Image.fromarray(pixels)
    encoded.save(output_image_path)
    
    end_time = time.time()
    encoding_time = end_time - start_time
    
    return "Encoding completed successfully.", encoding_time

if __name__ == "__main__":
    cover_image_path = 'cover_image.jpg'
    bin_file_path = 'example.bin'
    output_image_path = 'encoded_image0002.png'

    binary_message = read_binary_file(bin_file_path)
    result, encoding_time = encode_binary_image(cover_image_path, binary_message, output_image_path)
    print(result)
    print("Encoding time:", encoding_time, "seconds")


Encoding completed successfully.
Encoding time: 0.34256863594055176 seconds


In [43]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from PIL import Image
import time
import os

def decode_binary_image(encoded_image_path, output_bin_path):
    start_time = time.time()
    img = Image.open(encoded_image_path)
    width, height = img.size
    pixels = np.array(img, dtype=np.uint8)
    binary_message = np.zeros(width * height * 3 * 4, dtype=np.uint8)  # Max possible size

    mod = SourceModule("""
    __global__ void decode_kernel(unsigned char *pixels, unsigned char *binary_message, int message_length) {
        int idx = blockIdx.x * blockDim.x + threadIdx.x;
        int bit_idx = idx * 4;
        if (bit_idx < message_length) {
            for (int n = 0; n < 3; ++n) {
                unsigned char channel = pixels[idx * 3 + n];
                binary_message[bit_idx + n * 4 + 0] = (channel >> 3) & 0x01;
                binary_message[bit_idx + n * 4 + 1] = (channel >> 2) & 0x01;
                binary_message[bit_idx + n * 4 + 2] = (channel >> 1) & 0x01;
                binary_message[bit_idx + n * 4 + 3] = channel & 0x01;
            }
        }
    }
    """)

    pixels_gpu = cuda.mem_alloc(pixels.nbytes)
    binary_gpu = cuda.mem_alloc(binary_message.nbytes)

    cuda.memcpy_htod(pixels_gpu, pixels)
    
    block_size = 512
    grid_size = (width * height + block_size - 1) // block_size

    decode_kernel = mod.get_function("decode_kernel")
    decode_kernel(pixels_gpu, binary_gpu, np.int32(len(binary_message)), block=(block_size, 1, 1), grid=(grid_size, 1, 1))

    cuda.memcpy_dtoh(binary_message, binary_gpu)

    # Extract message length
    length_binary = ''.join(map(str, binary_message[:32]))
    message_length = int(length_binary, 2)

    binary_message = binary_message[32:32 + message_length]

    # Convert the binary message to bytes
    binary_data = int(''.join(map(str, binary_message[:message_length])), 2).to_bytes((message_length + 7) // 8, byteorder='big')

    with open(output_bin_path, 'wb') as file:
        file.write(binary_data)

    end_time = time.time()
    decoding_time = end_time - start_time

    return "Decoding completed successfully.", decoding_time

if __name__ == "__main__":
    encoded_image_path = 'encoded_image0002.png'
    output_bin_path = 'decoded_message195656127.bin'

    result, decoding_time = decode_binary_image(encoded_image_path, output_bin_path)
    print(result)
    print("Decoding time:", decoding_time, "seconds")


Decoding completed successfully.
Decoding time: 0.08441638946533203 seconds


The binary files are identical.
